# Explorando la API de DBLP: Introducción y objetivos

## ¿Qué es DBLP?
DBLP es una base de datos bibliográfica en línea que proporciona información sobre publicaciones científicas, principalmente en el ámbito de la informática. Esta API permite acceder a datos estructurados sobre artículos, autores y conferencias, facilitando la búsqueda y el análisis de publicaciones académicas.

[Sitio web oficial de DBLP](https://dblp.org/)

---

- ¿Qué problemas puede enfrentar una API si no establece límites de consumo?
- ¿Cómo podría afectar esto a los usuarios finales o a los desarrolladores?

---



## ¿Cómo consumiríamos la API sin una especificación clara?
La API de DBLP no tiene una especificación formal sobre sus límites de consumo (*rate limits*) o cuotas. Sin embargo, en el siguiente [FAQ de DBLP](https://dblp.org/faq/Am+I+allowed+to+crawl+the+dblp+website.html), se indica que las solicitudes deben espaciarse cada 2 segundos. Seguimos sin un límite claro de solicitudes por minuto, pero tras realizar pruebas, se ha observado que el límite asumible es de **5 solicitudes por minuto**.

Vamos a probar un consumo regulado durante **1 minuto**, espaciando las solicitudes cada 3 segundos. Si el límite asumido de **5 solicitudes por minuto** es correcto, no deberíamos obtener errores 429.

In [1]:

import yaml
from Pricing4API.ancillary.yaml_serialization import PricingYamlHandler
from Pricing4API.ancillary.limit import Limit
from Pricing4API.ancillary.time_unit import TimeDuration, TimeUnit
from Pricing4API.main.plan import Plan
from Pricing4API.main.subscription import Subscription
PricingYamlHandler.load()

In [2]:
dblp_yaml = """
!Pricing
DBLP Pricing:
  metrics:
    name: requests
  plans:
    Free DBLP:
      cost: 0.0
      billing_cycle:
        value: 1
        unit: month
      unitary_rate: 
        /*:
          all:
            requests:
              period:
                value: 3
                unit: second
      quotas:
        /*:
          all:
            requests:
              - max: 5
                period:
                  value: 1
                  unit: minute
"""



In [ ]:
pricing_DBLP = yaml.safe_load(dblp_yaml)

planes = pricing_DBLP.plans

for index, plan in enumerate(planes):
    print(plan.name," index: " , index)

In [ ]:
free_dblp = planes[0] #type: Plan
dblp_subscription = Subscription(free_dblp, 'https://dblp.org/search/publ/api')
dblp_subscription.api_usage_simulator(TimeDuration(1, TimeUnit.MINUTE))

¿Obtenemos el resultado esperado? Para entenderlo mejor vamos a introducir el concepto de curva de capacidad.

---

- ¿Qué es una curva de capacidad?
- ¿Cómo se relaciona con los límites de consumo de una API?

---

Es una representación del número máximo de solicitudes que podemos realizar a lo largo del tiempo, considerando las cuotas del plan y los intervalos de tiempo establecidos. Nos muestra cuántas solicitudes podríamos haber consumido en un momento dado si respetamos estrictamente los límites.


In [ ]:
free_dblp.show_available_capacity_curve(TimeDuration(1, TimeUnit.MINUTE))

**¿Hemos respetado los límites de consumo de la API de DBLP?**

En la simulación previa no hemos obtenido errores 429, aún así para realmente garantizar que no hwmoa excedido los límites, la curva de demanada (el consumo real de las solicitudes) siempre debe mantenerse por **debajo o igual** a la curva de capacidad ideal. De lo contrario, podríamos exceder los límites de consumo y recibir errores 429.



In [ ]:
dblp_subscription.demand_curve_vs_ideal_capacity()

Definir la vigencia de un Pricing y la documentación actualizada son aspectos claves para garantizar un consumo adecuado de la API.

- Sin una especificación clara y actualizada, los desarrolladores deben deducir los límites mediante pruebas, lo que puede generar ineficiencias y errores.

## Regulado vs. No regulado

Cuando la suscripción está regulada, el simulador respeta los límites establecidos en el plan, asegurando que no aparezcan errores `429 Too Many Requests`.

Sin embargo, si marcamos la suscripción como no regulada, el simulador intentará realizar las solicitudes tan rápido como sea posible, ignorando los límites establecidos. En este caso, observaremos errores `429` en pocos segundos, ya que la API detectará el exceso de consumo.

In [ ]:
dblp_subscription.regulated(False)

dblp_subscription.api_usage_simulator(TimeDuration(30, TimeUnit.SECOND))

## ¿Y si tras un consumo no regulado, intento hacerlo de forma regulada justo después?

Como hemos observado en las pruebas anteriores, si intentamos consumir la API de forma no regulada, rápidamente nos encontraremos con errores `429 Too Many Requests`. Esto se debe a que estamos excediendo los límites de consumo permitidos por la API.

Ahora bien, si después de recibir un error `429` intentamos volver a consumir la API de forma regulada, nos encontraremos con que no es posible hacerlo inmediatamente. Esto se debe a que debemos esperar a que pase el tiempo especificado en la cabecera de "cooling down period" antes de poder realizar nuevas solicitudes de forma regulada.


In [ ]:
dblp_subscription.regulated(True)

dblp_subscription.api_usage_simulator(TimeDuration(30, TimeUnit.SECOND)) 

In [ ]:
# import time
# time.sleep(300)
# dblp_subscription.api_usage_simulator(TimeDuration(1, TimeUnit.MINUTE))


## ¿Cómo afectaría un error 429 en la capacidad disponible?

Para entender este concepto pongamos un caso en el que teniendo la misma simulación anterior, de 1 minuto. Recibo un error en la segunda solicitud.

In [ ]:
mock_429_errors = [(2, '300')] # 300 segundos de coolingdown en la segunda solicitud.

dblp_subscription.plot_combined_capacity_curves(mock_429_errors)


---

## ¿Cuántas solicitudes puedes hacer?

Basándonos en lo que sabemos sobre la API, intenta responder a las siguientes preguntas. Usa los valores de **rate** y **cuota** para responder:

1. ¿Cuántas solicitudes se pueden hacer en:
   - 10 segundos?
   - 1 minuto?
   - 1 minuto y 1 segundo?

---

Para responder con precisión a estas preguntas, necesitamos una función que calcule el número máximo de solicitudes permitidas en un período \( t \), respetando tanto el **rate** como la **cuota** del plan.


### Ejercicio: Implementar capacity(t)

La función `capacity(t)` ya está implementada en la clase `Plan`. Vamos a probarla en distintos escenarios:

In [ ]:
time_simulation_10s = TimeDuration(1, TimeUnit.SECOND)
time_simulation_1m = TimeDuration(1, TimeUnit.MINUTE)
time_simulation_1min_1s = TimeDuration(1, TimeUnit.MINUTE) + TimeDuration(1, TimeUnit.SECOND)

capacity_10s = free_dblp.available_capacity(time_simulation_10s, len(free_dblp.limits) - 1)
capacity_1m = free_dblp.available_capacity(time_simulation_1m, len(free_dblp.limits) - 1)
capacity_1min_1s = free_dblp.available_capacity(time_simulation_1min_1s, len(free_dblp.limits) - 1)

print(f"Capacidad en 10 segundos: {capacity_10s}")
print(f"Capacidad en 1 minuto: {capacity_1m}")
print(f"Capacidad en 1 minuto y 1 segundos: {capacity_1min_1s}")

## ¿Cuánto tiempo necesitas para hacer X solicitudes?

Usando los límites de la API (rate y cuota), intenta calcular cuánto tiempo tomaría realizar las siguientes solicitudes. Piensa si el rate o la cuota influirán más en el resultado:

1. ¿Cuánto tiempo necesitas para realizar:
   - 5 solicitudes?
   - 6 solicitudes?
   - 20 solicitudes?

---

Para responder estas preguntas, necesitamos una función, que calcule el tiempo mínimo necesario para realizar un número dado de solicitudes, respetando tanto el **rate** como la **cuota** del plan.

### Ejercicio: min_time(capacity_goal)

La función `min_time(capacity_goal)` ya está implementada en la clase `Plan`. Vamos a probarla en distintos escenarios:

In [ ]:
# Tiempo necesario para realizar 10, 20 y 100 solicitudes
time_5_requests = free_dblp.min_time(5)
time_6_requests = free_dblp.min_time(6)
time_20_requests = free_dblp.min_time(20)

print(f"Tiempo para 5 solicitudes: {time_5_requests}")
print(f"Tiempo para 6 solicitudes: {time_6_requests}")
print(f"Tiempo para 20 solicitudes: {time_20_requests}")